If I have time, I would go over all the null values again, but I'm not sure

Things to do for cleanup:
- Go through all the data in a folder, and record the apartment complex with null data.
- Delete the rows corresponding to those apartment complexes in all of the data.
- Bring the apartment information, and get the corresponding number of households.
- Arrange it in the order of the rows in the energy usage data.
- Divide the corresponding values. 

Columns I need to keep: kaptCode, Date, electP, electC
- Then I should get all the apartments separately, and make a time-series dataframe out of all of them,(12+12 = 24 items? this is way too small... but I gotta try)
- Maybe I can focus on a specific Gu for now? 
- I need lots of data!
- I also need to get variables from apartment -> main exposure = electricity contract, confounders = elevator number, number of parking lot spaces, begin date

In [1]:
import pandas as pd
import numpy as np
import os

In [96]:
region = [
#     'Dobong',
#  'Dongdaemun',
#  'Dongjak',
#  'Eunpyeong',
#  'Gangbuk',
#  'Gangdong',
#  'Gangnam',
#  'Gangseo',
 'Geumcheon',
 'Guro',
 'Gwanak',
 'Gwangjin',
 'Jongno',
 'Jungnang',
 'Jung',
 'Mapo',
 'Nowon',
 'Seocho',
 'Seodaemun',
 'Seongbuk',
 'Seongdong',
 'Songpa',
 'Yangcheon',
 'Yeongdeungpo',
 'Yongsan']

In [102]:
import re

In [109]:
for city in region:    
    csv_list = os.listdir(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptEnergy/{city}_Gu_Only')
    test_df = pd.read_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptEnergy/{city}_Gu_Only/{csv_list[0]}", index_col=0)
    codeDict = test_df['kaptCode'].to_dict()
    info_df = pd.read_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/{city}_Gu_Only.csv')

    # Make dataframes for each apartment id
    n = len(test_df)
    dfLists = [[] for _ in range(n)]

    for date in csv_list:
        date_df = pd.read_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptEnergy/{city}_Gu_Only/{date}', index_col = 0)
        for i in range(len(date_df)):
            if(len(date)> 8):
                p = re.compile('[0-9]{6}')       
                date_temp = p.findall(csv_list[-1])[0]
                dat = np.zeros(len(date_df))
                dat.fill(int(date_temp))
                date_df['Date'] = dat
            cut_df = date_df[date_df['kaptCode'] == date_df['kaptCode'][i]][['kaptCode','Date', 'electC', 'electP']]
            dfLists[i].append(cut_df)

    for i in range(len(dfLists)):
        dfLists[i] = pd.concat(dfLists[i])

    useList = []
    nullList = []
    for i in range(len(dfLists)):
        if(dfLists[i].isnull().sum().sum() < 3):
            useList.append(i)
            nullList.append(dfLists[i].isnull().sum().sum())
    for i in useList:
        fillDict = dfLists[i][['electC', 'electP']].mean().to_dict()
        dfLists[i].fillna(fillDict, inplace = True)

    # bring apartment codes from city
    df1 = info_df[['kaptCode', 'Elec_Con']].copy()
    df1['Elec_Con'] = df1['Elec_Con'].replace({'종합계약': 1, '단일계약': 0, '기타': 0})
    df1['Elec_Con'].value_counts()
    # make electricity contract binary
    df11 = []
    for i in useList:  
        df11.append(df1[df1['kaptCode'] == codeDict[i]])

    df11 = pd.concat(df11)

    newdfList = []
    # Divide values by number of households

    for i in useList:
        df = dfLists[i]
        num = int(info_df[info_df['kaptCode'] == codeDict[i]]['Household_Num'])
        new_df = pd.DataFrame()
        new_df['kaptCode'] = df['kaptCode']
        new_df['Date'] = df['Date']
        new_df['electC'] = df['electC'] / num
        new_df['electComb'] = (df['electC'] + df['electP']) / num
        newdfList.append(new_df)


    ultdf = pd.concat(newdfList)

    directory = f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}'

    if not os.path.exists(directory):
        os.makedirs(directory)
        ultdf.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_MY.csv')
        df11.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_A.csv')
        print("Directory created.")
    else:
        ultdf.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_MY.csv')
        df11.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_A.csv')
    
    

Directory created.
Directory created.
Directory created.
Directory created.
Directory created.
Directory created.
Directory created.
Directory created.
Directory created.
Directory created.
Directory created.


In [87]:

csv_list = os.listdir('C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptEnergy/Dobong_Gu_Only')
test_df = pd.read_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptEnergy/{city}_Gu_Only/202212", index_col=0)
codeDict = test_df['kaptCode'].to_dict()
info_df = pd.read_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/{city}_Gu_Only.csv')

In [88]:
# Make dataframes for each apartment id
n = len(test_df)
dfLists = [[] for _ in range(n)]

for date in csv_list:
    date_df = pd.read_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptEnergy/{city}_Gu_Only/{date}', index_col = 0)
    for i in range(len(date_df)):
        cut_df = date_df[date_df['kaptCode'] == date_df['kaptCode'][i]][['kaptCode','Date', 'electC', 'electP']]
        dfLists[i].append(cut_df)

for i in range(len(dfLists)):
    dfLists[i] = pd.concat(dfLists[i])

useList = []
nullList = []
for i in range(len(dfLists)):
    if(dfLists[i].isnull().sum().sum() < 3):
        useList.append(i)
        nullList.append(dfLists[i].isnull().sum().sum())
for i in useList:
    fillDict = dfLists[i][['electC', 'electP']].mean().to_dict()
    dfLists[i].fillna(fillDict, inplace = True)

newdfList = []
# Divide values by number of households

for i in useList:
    df = dfLists[i]
    num = int(info_df[info_df['kaptCode'] == codeDict[i]]['Household_Num'])
    new_df = pd.DataFrame()
    new_df['kaptCode'] = df['kaptCode']
    new_df['Date'] = df['Date']
    new_df['electC'] = df['electC'] / num
    new_df['electComb'] = (df['electC'] + df['electP']) / num
    newdfList.append( new_df)


ultdf = pd.concat(newdfList)



In [89]:
print(len(dfLists), len(useList))

108 93


table 1
- index
- apartment codes
- treatment (contract yes or no)

table 2
- apartment codes
- date
- electricity values
then group by apartment codes and date? 

In [90]:
# bring apartment codes from city
df1 = info_df[['kaptCode', 'Elec_Con']].copy()
df1['Elec_Con'] = df1['Elec_Con'].replace({'종합계약': 1, '단일계약': 0, '기타': 0})
df1['Elec_Con'].value_counts()
# make electricity contract binary
df11 = []
for i in useList:  
    df11.append(df1[df1['kaptCode'] == codeDict[i]])

df11 = pd.concat(df11)

In [91]:
newdfList = []
# Divide values by number of households

for i in useList:
    df = dfLists[i]
    num = int(info_df[info_df['kaptCode'] == codeDict[i]]['Household_Num'])
    new_df = pd.DataFrame()
    new_df['kaptCode'] = df['kaptCode']
    new_df['Date'] = df['Date']
    new_df['electC'] = df['electC'] / num
    new_df['electComb'] = (df['electC'] + df['electP']) / num
    newdfList.append( new_df)


ultdf = pd.concat(newdfList)

In [92]:
directory = f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}'

if not os.path.exists(directory):
    os.makedirs(directory)
    ultdf.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_MY.csv')
    df11.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_A.csv')
    print("Directory created.")
else:
    ultdf.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_MY.csv')
    df11.to_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{city}/{city}_A.csv')

Directory created.


In [110]:
region = [
    'Dobong',
 'Dongdaemun',
 'Dongjak',
 'Eunpyeong',
 'Gangbuk',
 'Gangdong',
 'Gangnam',
 'Gangseo',
 'Geumcheon',
 'Guro',
 'Gwanak',
 'Gwangjin',
 'Jongno',
 'Jungnang',
 'Jung',
 'Mapo',
 'Nowon',
 'Seocho',
 'Seodaemun',
 'Seongbuk',
 'Seongdong',
 'Songpa',
 'Yangcheon',
 'Yeongdeungpo',
 'Yongsan']

In [115]:
# Giving up on the time series and just doing 202210
# Get all 202210 points for Seoul
Seoul = []

# Get the values from each region
for r in region:
    df_MY= pd.read_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{r}/{r}_MY.csv', index_col = 0)
    df_A = pd.read_csv(f'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/{r}/{r}_A.csv', index_col = 0)
    df_MY = df_MY[df_MY['Date'] == 202210]
    df = pd.merge(df_MY, df_A, on= 'kaptCode')
    Seoul.append(df)

Seoul_df = pd.concat(Seoul)
final_path = 'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/Seoul/Final.csv'
Seoul_df.to_csv(final_path)

In [114]:
Seoul_df

,kaptCode,Date,electC,electComb,Elec_Con
0,A13203302,202210.0,12384.995086,46682.882064,0.0
1,A13203303,202210.0,4845.243932,30853.593447,0.0
2,A13295201,202210.0,12872.025806,49366.993548,0.0
3,A13286107,202210.0,770.257732,41554.158076,0.0
4,A13287801,202210.0,5637.128920,41772.599303,0.0
...,...,...,...,...,...
40,A10025192,202210.0,124528.275660,334050.240469,1.0
41,A14024001,202210.0,29379.651163,71962.674419,0.0
42,A14024002,202210.0,12461.078431,56840.558069,0.0
43,A14082601,202210.0,9828.584071,45951.238938,0.0


In [153]:
Only_df = pd.read_csv('C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/Seoul_Only.csv')
Only_df = Only_df[['kaptCode','Elev_Num', 'Park_Below', 'Begin_Date', 'Household_Num']].dropna()
result = pd.merge(Seoul_df, Only_df, how = 'left', on = 'kaptCode')
result = result.dropna()
result['Elev_Num'] = result['Elev_Num'] / result['Household_Num']
result['Park_Below'] = result['Park_Below'] / result['Household_Num']
result = result.drop('Household_Num', axis = 1)
result['Begin_Date'] = pd.to_datetime(result['Begin_Date'], format = "%Y%m%d")
result['Begin_Date'] = result['Begin_Date'].max() - result['Begin_Date'] 
result['Begin_Date'] = result['Begin_Date'].dt.days

In [154]:
result

,kaptCode,Date,electC,electComb,Elec_Con,Elev_Num,Park_Below,Begin_Date
0,A13203302,202210.0,12384.995086,46682.882064,0.0,0.036855,0.850123,7146
1,A13203303,202210.0,4845.243932,30853.593447,0.0,0.019417,0.000000,12861
2,A13295201,202210.0,12872.025806,49366.993548,0.0,0.038710,0.941935,5532
3,A13286107,202210.0,770.257732,41554.158076,0.0,0.037801,0.254296,7268
4,A13287801,202210.0,5637.128920,41772.599303,0.0,0.045296,0.080139,9750
...,...,...,...,...,...,...,...,...
1500,A10025192,202210.0,124528.275660,334050.240469,1.0,0.161290,4.674487,827
1501,A14024001,202210.0,29379.651163,71962.674419,0.0,0.029070,1.401163,7066
1502,A14024002,202210.0,12461.078431,56840.558069,0.0,0.038462,0.000000,13748
1503,A14082601,202210.0,9828.584071,45951.238938,0.0,0.039823,0.353982,10743


In [148]:
final_path = 'C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/Stat_Analysis/Data_Cleanup/Data_Stat/Seoul/Final.csv'
result.to_csv(final_path)

In [134]:
Only_df[['kaptCode','Elev_Num', 'Park_Below', 'Begin_Date']]
# pd.to_datetime(['Begin_Date'], format = "%Y%m%d").dt.normalize()

,kaptCode,Elev_Num,Park_Below,Begin_Date
0,A13203302,15.0,346.0,20020726.0
1,A13203303,16.0,0.0,19861202.0
2,A13295201,6.0,146.0,20061226.0
3,A13286107,22.0,148.0,20020326.0
4,A13287801,13.0,23.0,19950609.0
...,...,...,...,...
2623,A13681701,7.0,179.0,19940624.0
2624,A13676101,19.0,684.0,20090626.0
2625,A13676102,2.0,56.0,20090626.0
2626,A13615002,10.0,357.0,19990612.0
